# Stability and Chaos in Networks of Spiking Neurons

Introduction In the theory of dynamical systems a central question is whether small perturbations to a system’s trajectory diminish, i.e. the perturbed trajectory evolves as unperturbed, or whether initially small perturbations grow. The former expresses stability to perturbations. On the other side, if perturbations grow exponentially fast, irrespective of the initial condition, the system is chaotic. Both, stability and chaos, can be desirable for neural networks; stability is useful for memory and reliability, chaos is useful for distinction and classification.

Network of leaky integrate-and-fire neurons.

The parameters are $N = 10 000, K = 1000, \bar \nu = 10 \mathrm{Hz}, J_0 = 1, \tau_m= 10 \mathrm{ms}$.

In [ ]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt

### Repro & simulation dt

In [ ]:
np.random.seed(0)
seed(0)

defaultclock.dt = 0.1*ms

### Output folder

In [ ]:
from pathlib import Path

path_results = Path("results")
path_results.mkdir(parents=True, exist_ok=True)

## Parameter

In [ ]:
T = 5.0*second

N       = 10_000    # number of neurons
K       = 1_000     # connectivity - each neuron connected to K different neurons

V_T     = 1.0       # voltage threshold
V_R     = 0.0       # voltage reset

nu      = 10*Hz     # firing rate
J_0     = 1.0
tau_m   = 10*ms

In [ ]:
# Balanced-state scaling (eq. (5)): nu ~ nu_bal = I_0/(J_0*tau_m)  -> I_0 ~ nu*J_0*tau_m
I_0 = nu * J_0 * tau_m

Iext_const = np.sqrt(K) * I_0   # constant excitatory current
w_inh = J_0/np.sqrt(K)          # inhibitory input received

## Task1

In [ ]:
eqs = """
dV/dt = (-V + I_ext)/tau_m : 1
I_ext : 1
"""

G = NeuronGroup(
    N,
    model=eqs,
    threshold="V >= V_T",
    reset="V = V_R",
    method="euler"
)

G.V = "rand() * V_T"
G.I_ext = Iext_const

### Connectivity (Erdős–Rényi)

In [ ]:
p = K / (N-1)
S = Synapses(G, G, on_pre="V -= w_inh")  # inhibitory undelayed pulse 
S.connect(condition='i!=j', p=p)

### Monitors & run

In [ ]:
n_plot = 30
idx = np.random.choice(N, size=n_plot, replace=False)
i_0 = int(idx[0])

sp_mon = SpikeMonitor(G)
v_mon = StateMonitor(G, "V", record=[i_0])

print(f"Running simulation for T = {T} ...")
run(T)
print("Done!")


### Plots

In [ ]:
import seaborn as sns

(a) Asynchronous irregular spike pattern of 30 randomly chosen neurons.

In [ ]:
# Spike raster
fig, ax = plt.subplots(figsize=(10, 4))
idx_sorted = np.sort(idx)

trains = sp_mon.spike_trains()
spike_lists = [(trains[int(n)]/ms) for n in idx_sorted]

ax.eventplot(spike_lists, colors="black", linewidths=0.8)
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Neuron (subset index)")
ax.set_xlim(0, float(T/ms))
sns.despine()
plt.tight_layout()
plt.savefig(path_results / "task1_spike_raster.png", dpi=150, bbox_inches="tight")
plt.show()

(b) Fluctuating voltage trace of one neuron.

In [ ]:
# Membrane voltage
fig, ax = plt.subplots(figsize=(10, 4))

ax.plot(v_mon.t/ms, v_mon.V[0], linewidth=1.0, label='V(t)')
ax.axhline(V_T, color='red', linestyle='--', linewidth=1.5, label=f'$V_T$ = {V_T}')
ax.axhline(V_R, color='gray', linestyle=':', linewidth=1.0, alpha=0.5, label=f'$V_R$ = {V_R}')

# Spike marker
spike_times_i0 = sp_mon.t[sp_mon.i == i_0] / ms
spike_vals = np.ones_like(spike_times_i0) * V_T
ax.plot(spike_times_i0, spike_vals, marker='o', linestyle='none', 
        color='red', markersize=5, label='Spike events', zorder=5)

ax.set_xlabel("Time [ms]")
ax.set_ylabel("V [mV]")
#ax.set_title(f"(b) Membrane voltage: Neuron {i_0}", fontsize=12, loc='left')
ax.set_xlim(0, float(T/second)*1000)
ax.set_ylim(-0.2, 1.5)
ax.legend(loc='upper right', fontsize=9)
sns.despine()
plt.tight_layout()
plt.savefig(path_results / "task1_membrane_voltage.png", dpi=150, bbox_inches='tight')
plt.show()


(c) Broad distributions of individual neurons’ firing rates $\bar\nu$ and coefficients of variation cv. 

In [ ]:
# Mean firing rate per neuron over whole simulation
spike_counts = np.bincount(sp_mon.i, minlength=N)  # spikes per neuron
rates_hz = spike_counts / float(T / second)  # Hz

fig, ax = plt.subplots(figsize=(7, 4))

if len(rates_hz) > 1:
    sns.kdeplot(
        x=rates_hz,
        ax=ax,
        bw_adjust=0.3,
        cut=0,
        clip=(0, None),
        linewidth=2
    )
    ax.axvline(np.mean(rates_hz), color="red", linestyle="--",
               label=f"Mean: {np.mean(rates_hz):.2f} Hz")
    ax.legend()

ax.set_xlabel("Firing rate [Hz]")
ax.set_ylabel("Density")

sns.despine()
plt.tight_layout()
plt.savefig(path_results / "task1_rate_dist.png",
            dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
N = sp_mon.source.N
T_sec = float(T / second)

cv = np.full(N, np.nan, dtype=float)

trains = sp_mon.spike_trains()  # dict: neuron_index -> array of spike times

for n in range(N):
    t_n = trains[n]
    if len(t_n) >= 3:
        isi_n = np.diff(np.sort(t_n)) / second
        m = np.mean(isi_n)
        s = np.std(isi_n)
        if m > 0:
            cv[n] = s / m

cv_valid = cv[np.isfinite(cv)]

fig, ax = plt.subplots(figsize=(7, 4))

if len(cv_valid) > 1:
    sns.kdeplot(
        x=cv_valid,
        ax=ax, 
        bw_adjust=0.4, 
        cut=0, 
        clip=(0, None),
        linewidth=2
    )
    ax.axvline(np.mean(cv_valid), color="red", linestyle="--",
                    label=f"Mean: {np.mean(cv_valid):.2f}")
    ax.legend()

ax.set_xlabel("CV of ISI (std/mean)")
ax.set_ylabel("Density")

sns.despine()
plt.tight_layout()
plt.savefig(path_results / "task1_rate_cv_dist.png",
            dpi=150, bbox_inches="tight")
plt.show()


## Task 2

In [ ]:
def T_free(tau = tau_m, VT=V_T, VR=V_R, I_ext=Iext_const):
    '''a constant value needed for voltage to phase space transition
    '''
    ln = np.log((VT-I_ext)/(VR-Iext_const))
    return -tau * ln

def phi(V):
    '''
    takes unitless Voltage value, returns phi space
    '''
    Tfree = T_free(tau = tau_m, VT=V_T, VR=V_R, I_ext=Iext_const)
    return - tau_m / Tfree *(np.log((V-Iext_const)/(V_R-Iext_const)))

def phi_inv(phi, C = Iext_const, tau_m = tau_m):
    '''Converts from phi space from voltage space
    '''
    Tfree = T_free(tau = tau_m, VT=V_T, VR=V_R, I_ext=Iext_const)/ms
    exponent = - phi * Tfree / (tau_m/ms)
    return (V_R - C) * np.exp(exponent) + C

## Task 3, 4, 5

In [ ]:
def run_w_startvector(runtime, startvector):   #enter the runtime in seconds but with the unit
    
    start_scope() 
    defaultclock.dt = 0.1*ms

    # makes use of all variables declared globally, i.e. V_f, V_t, nu, N, K...

    # --- Neuronmodell (Voltage-LIF) ---
    # tau_m dV/dt = -V + I(t) (1), with I(t)=sqrt(K)*I_0 - (J_0/sqrt(K))*sum delta(...)
    eqs = """
    dV/dt = (-V + I_ext)/tau_m : 1  #dimensionless
    I_ext : 1                       #dimensionless constant excitatory current
    """

    G = NeuronGroup(
        N,
        model=eqs,
        threshold="V >= V_T",
        reset="V = V_R",
        method="euler"
    )

    # Initialisierung
    G.V = startvector   #initialize voltage values to start vector
    G.I_ext = Iext_const   #initialize excitatory current with global value

    #connectivity probability
    p = K / (N-1)   # p in (0,1)

    seed(0)           # Brian2 seed
    S = Synapses(G, G, on_pre="V -= w_inh")  # when action potential occurs, decrease the voltage of post-synaptic neuron
    S.connect(condition='i!=j', p=p) # set recurrent connections with probability p, set all self recurrent connections to 0


    #sp_mon = SpikeMonitor(G)
    mon_all_V = StateMonitor(G, 'V', record=True) #((len(indices), len(t)))

    print(f"\nRunning simulation for T = {runtime}")

    run(runtime)
    mon_endvalues = np.array(G.V) #takes only final voltages

    return mon_endvalues, mon_all_V



In [ ]:
# initial random vector of size 10k, a voltage value [0,1) for all 10k neurons
initial_randvector = np.random.rand(10000)

# run for 1s
runtime = 1*second
# V_0 is the voltage vector after 1second free run
V_0, trajectory_V_0  = run_w_startvector(runtime, initial_randvector)


In [ ]:
#run V_0 for 10ms
runtime = 10*ms
V_1, trajectory_V_1  = run_w_startvector(runtime, V_0)

Helpful info. <br>
V_0: (N,). <br>
trajectory_V_0.V: (N,runtime) // runtime is in 0.1ms's. 


In [ ]:
#convert to phase
phase_V_0 = phi(V_0) # (N,), same dimension

#create and add the random vector (perturb in phase space)
r = np.random.normal(0,1,N)
r /= np.linalg.norm(r)
phase_V_0_shift = phase_V_0 + r

#convert the perturbed phase vector back to voltage space
# this means we slightly altered the V_0
V_0e = phi_inv(phase_V_0_shift)

In [ ]:
#run from the slighly altered starting point V0_e
V_1e, trajectory_V_1e = run_w_startvector(runtime, V_0e)

# calculate difference of voltage vectors in phase space between original V0 and altered V0e
# trajectories.V: (N,runtime), we subtract each entries of the (N,runtime) matrix
phase_V_5_shift=phi(trajectory_V_1.V)
phase_V_5e_shift=phi(trajectory_V_1e.V)
diff_eps_phase = np.abs(phase_V_5_shift - phase_V_5e_shift)

# sum the difference over all neurons per timestep and normalize by dividing to N
# final vector: (runtime,)
diff_over_neurons_normalized = diff_eps_phase.sum(axis=0) / N


In [ ]:
def plot_time_difference(diff_over_neurons_normalized,runtime):
    '''
    plot voltage difference against time
    '''
    plt.plot(diff_over_neurons_normalized)
    plt.yscale('log')
    plt.xlabel(f'runtime {runtime} / timestep (0.1ms)')
    plt.ylabel(f'average phase difference of 10k neurons')
    plt.title(r'Time dependent difference $D_{\phi}(t) = \|\phi(V(t)) - \phi(V_{\varepsilon}(t))\|$')
    plt.legend()

In [ ]:
plot_time_difference(diff_over_neurons_normalized,10*ms) # 1*s

plt.savefig(path_results / "task3_D_phi.png", dpi=150, bbox_inches="tight")

3. <br>
In this particular perturbation (e=1), after running V1 and V1e for 10ms seconds (100 timesteps), the time dependent difference between two initial voltage values of the neurons increase and stabilize around 0.2 mV. This means that the perturbed initial voltage starting point finds stability in a flux tube different than the unperturbed voltage.

In [ ]:
# find an epsilon that decreases (converges in the same flux tube), and an epsilon that increases.
# we already have our normalized random vector r = np.random.normal(0,1,N),  r /= np.linalg.norm(r)
# so we scale it up and down and try to find bifurcation values for epsilon in our system 
# - leading to convergence of the same flux tube vs different flux tube

In [ ]:
def simulate_epsilon_differences(eps_array,num_realizations,V0,runtime):
    phase_V0=phi(V0)
    # run unperturbed vector V0
    V1, trajectory_V1 = run_w_startvector(runtime, V0)
    phase_V1_shift=phi(trajectory_V1.V)
        
    divergence_probabilities = []

    for eps in eps_array:
        divergence_count = 0

        for realization in range(num_realizations):
            # add perturbation to V0
            np.random.seed(realization)
            r1 = np.random.normal(0,1,N)
            r1 /= np.linalg.norm(r1)
            phase_V0e_shift = phase_V0 + (eps*r1)
            V0e=phi_inv(phase_V0e_shift)
            
            # run the perturbed vector V0e
            print(f'epsilon = {eps}')
            V1e, trajectory_V1e = run_w_startvector(runtime, V0e)
            
            # calculate difference between trajectories of original and perturbed vectures
            phase_V1e_shift=phi(trajectory_V1e.V)
            diff_eps_phase = np.abs(phase_V1_shift - phase_V1e_shift)
            diff_over_neurons_normalized = diff_eps_phase.sum(axis=0) / N

            plt.figure()
            plot_time_difference(diff_over_neurons_normalized, runtime)
            plt.show()

            # calculate probability of distance between original and perturbed vectors increasing
            if diff_over_neurons_normalized[-1] > diff_over_neurons_normalized[0]:
                divergence_count += 1
                
        divergence_probabilities.append(divergence_count / num_realizations)

    return np.array(eps_array), np.array(divergence_probabilities)

In [ ]:
epsilon_array=[0.001,0.002,0.003,0.004,0.005]
simulate_epsilon_differences(epsilon_array,1,V_0,20*ms)

4. <br>
We observe that for e in [0.001,0.002], the difference in phase space between the original and perturbed vector decreases over time. This implies that a perturbation with e= 0.001 or e=0.002, for the initial location in the phase space is too small to exit the current flux tube. So the perturbed vector converges on the same stability as the original vector <br>
<br>
However,for e in [0.004,0.005], the difference in phase decreases over time. Such perturbations are strong enough to change the flux tube of the original vector. 
<br>
To be safe, we chose two epsilon values: e = 0.001 to demonstrate decreasing distance, e=0.01 to demonstrate increasing distance. We repeat the observation 3 times, with different initial random r vectors for both cases. 
<br>
We observe that for e=0.001 for all 3 of our simulations, the original and perturbed vectors converge to same vector. Whereas, for e=0.01, all 3 runs converge to different vectors.

In [ ]:
# reproduce epsilon = 0.001, and epsilon = 0.005 for 3 observations. 
epsilon_array1=[0.001] * 3
epsilon_array2=[0.01] * 3
simulate_epsilon_differences(epsilon_array1,1,V_0,20*ms)
simulate_epsilon_differences(epsilon_array2,1,V_0,20*ms)


5.

In [ ]:
# repeat with 15 values of e between 10^-5 and 10^0 (logscale separated)
# for 50 realizations per epsilon. Asses for each e, number of realizations where difference D_phi(t) doesn't go to zero. 
# i.e. (test final value of D_phi(t) different than D_phi(0))

eps_array = np.logspace(-5, 0, 15)
eps, prob = simulate_epsilon_differences(eps_array, 50, V_0, runtime)

plt.figure()
plt.semilogx(eps, prob, marker='o')
plt.xlabel(r'$\epsilon$')
plt.ylabel('Probability of divergence')
plt.title('Divergence probability vs perturbation size')
plt.grid(True, which='both')
plt.show()



In [ ]:
print(prob)
print(eps)

We observe that results look pretty similar to figure 5b. <br>
for epsilon <= 10^(-4), probability(observing divergence between original and perturbed vector)=0 <br>
for 0.001 < eps < 0.003, prob (observing divergence between original and perturbed vector) = 0.5 <br>
for eps > 0.015, prob = 1. 

6.

In [ ]:
# normalized random vectors p and q
N=200
np.random.seed(0)

p = np.random.normal(0,1,N)
p /= np.linalg.norm(p)

q = np.random.normal(0,1,N)
q /= np.linalg.norm(q)

In [ ]:
V_init_arr = np.zeros((20, 20, N))

for i in range(20):
    for j in range(20):
        V_init_arr[i, j, :] = (
            (-0.1 + i * 0.01) * p +
            (-0.1 + j * 0.01) * q
        )

# V_init_arr.shape (20, 20, 200)


In [ ]:
# simulate network run for 400 initial conditions. Compare difference between each pair of simulation i and i+1, and j and j+1, 
# check which pairs converge, which pairs diverge. Identify initial groups that converge to the same trajectory. In the plane of initial conditions, 
# color code every such group, and reproduce 7a. 

## Task 6

In [ ]:
# alter the function slightly to only output the endvector (for lower runtime)

def run_start_end(runtime, startvector):   #enter the runtime in seconds but with the unit
    
    start_scope() 
    defaultclock.dt = 0.1*ms

    # makes use of all variables declared globally, i.e. V_f, V_t, nu, N, K...

    # --- Neuronmodell (Voltage-LIF) ---
    # tau_m dV/dt = -V + I(t) (1), with I(t)=sqrt(K)*I_0 - (J_0/sqrt(K))*sum delta(...)
    eqs = """
    dV/dt = (-V + I_ext)/tau_m : 1  #dimensionless
    I_ext : 1                       #dimensionless constant excitatory current
    """

    G = NeuronGroup(
        N,
        model=eqs,
        threshold="V >= V_T",
        reset="V = V_R",
        method="euler"
    )

    # Initialisierung
    G.V = startvector   #initialize voltage values to start vector
    G.I_ext = Iext_const   #initialize excitatory current with global value

    #connectivity probability
    p = K / (N-1)   # p in (0,1)

    seed(2)           # Brian2 seed
    S = Synapses(G, G, on_pre="V -= w_inh")  # when action potential occurs, decrease the voltage of post-synaptic neuron
    S.connect(condition='i!=j', p=p) # set recurrent connections with probability p, set all self recurrent connections to 0


    #sp_mon = SpikeMonitor(G)
    # mon_all_V = StateMonitor(G, 'V', record=True) #((len(indices), len(t)))
    # popmon = PopulationRateMonitor(G)
    #print(f"\nRunning simulation for T = {runtime}")

    run(runtime)
    mon_endvalues = np.array(G.V) #takes only final voltages

    return mon_endvalues #, mon_all_V, popmon

In [ ]:
#starting from the center of a fluxtube

initial_randvector = np.random.rand(N)
#print(initial_randvector)

# V_0, trajectory, popmon = run_start_end(1*second, initial_randvector)
V_0 = run_start_end(0.5*second, initial_randvector)
print(V_0)

# plt.plot(trajectory.t , trajectory.V[0])
# plt.show()
# plt.plot(popmon.t , popmon.smooth_rate(width=0.5*ms))

In [ ]:
# Creating the two normalized random vectors p and q and adding them to the center of a fluxtube

p_vec = np.random.normal(0,1,N)
p_vec /= np.linalg.norm(p_vec)

q_vec = np.random.normal(0,1,N)
q_vec /= np.linalg.norm(q_vec)

# put this into a 20x20 matrix
Vij = np.zeros((20,20,N))

for a in range(0,20):
    for b in range(0,20):
        Vij[a,b] = (-0.01 + a * 0.001) * p_vec + (-0.01 + b * 0.001) * q_vec + V_0


In [ ]:
# run this for the 400 conditions
runtime = 10*ms

# create a matrix to store the resulting end vectors
# add a dimension of 1 to add the flux tube group later
end_matrix = np.zeros((20,20,1,N))


for a in range(0,20):
    print(f'Progress is on {a}/20.')
    for b in range(0,20):
        end_matrix[a,b,0] = run_start_end(runtime, Vij[a,b])


In [ ]:
for i in range(20):
       print(end_matrix[0,i,0])
#[0.62106411, 0.51349036, 0.08041475, ..., 0.10823177, 0.79704109,
       #0.57026998], shape=(10000,))

In [ ]:
# create an array which I can use to label the starting points into groups according to their trajectory
group_labels = np.zeros((20,20))

In [ ]:
end_matrix[0,19,0]

In [ ]:
threshold = 1.0  # adjust to your needs

for a in range(20):
    for b in range(20):
        if np.all(group_labels == 0):
            group_labels[a,b] = 1
        else:
            existing_groups = np.unique(group_labels)
            existing_groups = existing_groups[existing_groups != 0]
            
            assigned = False
            for g in existing_groups:
                indices = np.argwhere(group_labels == g)
                
                # Check ALL members of this group
                for i,j in indices:
                    diff = np.abs(end_matrix[a,b,0] - end_matrix[i,j,0]).sum()
                    if diff < threshold:
                        group_labels[a,b] = g
                        assigned = True
                        break
                if assigned:
                    break  # stop checking other groups
            
            if not assigned:
                # Only create new group if none matched
                group_labels[a,b] = int(np.max(group_labels) + 1)


In [ ]:
np.abs((end_matrix[0,0,0]-end_matrix[1,0,0])).sum()

In [ ]:
group_labels

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(group_labels, cmap='tab20', extent=[-0.01, 0.01, -0.01, 0.01], origin='lower')
plt.xlabel('b index')
plt.ylabel('a index')
plt.xticks([-0.01, 0, 0.01])
plt.yticks([-0.01, 0, 0.01])
plt.show()

plt.savefig(path_results / "task6_phasespace_structur.png", dpi=150, bbox_inches="tight")